In [459]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta
from dateutil.relativedelta import relativedelta

In [460]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\output data'
namedataASCD_M = r'dataASCD_M.xlsx'
rutedataASCD_M = os.path.join(folder,namedataASCD_M)

In [461]:
dataASCD_M = pd.read_excel(rutedataASCD_M,header=0)

C:\Users\jcallomamanib\AppData\Roaming\Python\Python38\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [462]:
filterMT = dataASCD_M.loc[:,'Maintenance type'] != 'COND'
dataASCD_1TM = dataASCD_M.loc[filterMT,:].reset_index(drop=True)

In [463]:
dataASCD_1TM.loc[:,'Strategy'] = dataASCD_1TM.loc[:,'Frequency'].astype(int).astype(str) + dataASCD_1TM.loc[:,'Unidad Periodo']

dataASCD_1TM.loc[:,'Man hour'] = dataASCD_1TM.loc[:,'Time']*dataASCD_1TM.loc[:,'Amount Craft']

In [464]:
# Adding the pivot column to project
dataASCD_1TM.loc[:,'pivot'] = dataASCD_1TM.loc[:,'key_dutie'] + '-' + dataASCD_1TM.loc[:,'Strategy'] 
#dataASCD_1TP.loc[:,'pivot'] = dataASCD_1TP.loc[:,'Asset tag'] + '-' + dataASCD_1TP.loc[:,'Strategy']

In [465]:
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo','Strategy', 'Date task done']

In [466]:
dataASCD_1TM.columns

Index(['Asset tag', 'Name', 'Cod sap', 'WBS_1_EN', 'WBS_2_EN', 'WBS_3_EN',
       'WBS_4_EN', 'Responsable', 'Hour day', 'Criticidad', 'Criticidad Name',
       'key_system', 'System', 'key_component', 'Component', 'key_dutie',
       'Primary task', 'Primary task detail activity', 'Maintenance type',
       'Specialist', 'Constraint', 'Time', 'Amount Craft', 'Frequency',
       'Unidad Periodo', 'Last date task done', 'Man hour', 'Strategy',
       'pivot'],
      dtype='object')

In [467]:
projectionM = dataASCD_1TM.loc[:,['key_dutie','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']].groupby(['key_dutie','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']).size().reset_index()
projectionM.drop(columns = {0},inplace =True)
projectionM.loc[:,'Date plan'] = projectionM.loc[:,'Last date task done']
[row,column] = projectionM.shape

In [468]:
projectionM.head()

,key_dutie,Strategy,Frequency,Unidad Periodo,Last date task done,Hour day,Date plan
0,2221-PU-201|01-04-01,165H,165.0,H,2021-03-30,24,2021-03-30
1,2221-PU-201|01-05-01,165H,165.0,H,2021-03-30,24,2021-03-30
2,2221-PU-201|01-06-01,165H,165.0,H,2021-03-30,24,2021-03-30
3,2221-PU-201|01-06-02,165H,165.0,H,2021-03-30,24,2021-03-30
4,2221-PU-201|01-06-03,4000H,4000.0,H,2021-03-27,24,2021-03-27


In [469]:
%%time
planDatesM = pd.DataFrame(columns = projectionM.columns)
projectionflag = projectionM.copy()
for m in range(row):
    while projectionflag.loc[m,'Date plan'] < date(2022,1,1):
        hour2days = round(projectionflag.loc[m,'Frequency']/projectionflag.loc[m,'Hour day'],0)
        if projectionflag.loc[m,'Unidad Periodo'] == 'H':
            delta = relativedelta(days = hour2days)
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'S':
            delta = relativedelta(weeks = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'M':
            delta = relativedelta(months = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'A':
            delta = relativedelta(years = projectionflag.loc[m,'Frequency'])
        projectionflag.loc[m,'Date plan'] = projectionflag.loc[m,'Date plan'] + delta
        if projectionflag.loc[m,'Date plan'] < date(2022,1,1):
            planDatesM = planDatesM.append(projectionflag.loc[m,:],ignore_index=True)
        else:
            pass
planDatesM.loc[:,'pivot'] = planDatesM.loc[:,'key_dutie'] + '-' + planDatesM.loc[:,'Strategy']

KeyboardInterrupt: 

In [ ]:
planDatesM

In [ ]:
planDatesM.shape

In [ ]:
%%time
dataASCD_1TMPlan = pd.merge(dataASCD_1TM,planDatesM.loc[:,['pivot','Date plan']],how='left',on='pivot')
dataASCD_1TMPlan.to_excel('dataASCD_1TMPlan.xlsx', encoding='utf-8-sig',index=False)

In [ ]:
dataASCD_1TMPlan.shape

In [ ]:
dataASCD_1TMPlan.columns